In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
from Tools import commonfxns as cf, OAPBuoyData as OAP, mplRC, OAPBuoyComp as bc, viz, evalfxns as ef,frequencyBands as fb
import netCDF4 as nc
import cftime
import datetime as dt
import cmocean
import gsw
import pickle
import scipy.stats as scst

%matplotlib inline
mplRC.paperRC2();
#mpl.rc('figure', dpi=200)

In [2]:
dfInfoBuoy=OAP.loadOAPInfo(modMeans=True)
#comps=bc.loadAllComps('/work/ebo/calcs/buoyCompTS/GFDL-ESM4.1.1975_2022/comps/')
dft=bc.loadStats(merged=True,path='/work/ebo/calcs/buoyCompTS/GFDL-ESM4.1.1975_2022/comps/').merge(dfInfoBuoy.loc[:,['datasetID','modBathy']],on=['datasetID'])

In [3]:
np.max(dft['modBathy'])

5686.166015625

In [4]:
def getstats(x0,y0):
    iii=(~np.isnan(x0))&(~np.isnan(y0))
    x=x0[iii];y=y0[iii]
    r,rp=scst.pearsonr(x,y)#,alternative='greater') default is 'two-sided'
    bias=np.nanmean(y)-np.nanmean(x)
    RMSE=np.sqrt(np.mean((y-x)**2))
    nRMSE=RMSE/np.std(x)
    Rsq=ef.Rsq(x,y)
    WSS=ef.WSS(x,y)
    N=len(x)
    return r,bias,RMSE,nRMSE,Rsq,WSS,N,rp

In [5]:
bc.dispNameUnits

{'phos': 'Surface pH',
 'phosC': 'Surface pH',
 'spco2': 'Surface pCO$_2$ (μatm)',
 'apco2': 'Surface Air pCO$_2$ (μatm)',
 'co2dryair': 'Air CO$_2$ (μmol mol$^{-1}$)',
 'tos': 'SST (°C)',
 'sos': 'SSS (psu)',
 'chlos': 'Surface Chl (μg l$^{-1}$)',
 'l10chlos': 'log$_{10}$[Chl (μg l$^{-1}$)]',
 'o2os': 'DO (μmol kg$^{-1}$)',
 'dpco2': '$\\Delta$ pCO$_2$ (μatm)',
 'hplusos': '[H$^{+}$] (μmol l$^{-1}$)',
 'MLD_003': 'Mixed layer depth (delta rho = 0.03) (m)',
 'intpp': 'Primary Organic Carbon Production by All Types of Phytoplankton (mol m-2 s-1)',
 'fgco2': 'Surface Downward Flux of Total CO2 (kg m-2 s-1)',
 'talkos': 'Surface Total Alkalinity (μmol kg$^{-1}$)',
 'dissicos': 'Surface Dissolved Inorganic Carbon Concentration (μmolkg$^{-1}$)',
 'dic_kw': 'Gas Exchange piston velocity for dic (m/sec)',
 'dic_sc_no': 'Ocean surface Schmidt Number for dic (mol/kg)',
 'pso': 'Sea Water Pressure at Sea Water Surface (Pa)',
 'friver': 'Water Flux into Sea Water From Rivers (kg m-2 s-1)',
 'nsmp

In [6]:
varlist=['tos','sos','phos','phosC','spco2','apco2','o2os','AOUos','chlos']#,'l10chlos','co2dryair']#,'o2perc','AOUos']

In [7]:
namedict={'tos':'SST (°C)','sos':'SSS (psu)','phos':'\sensorpH','phosC':'\calcpH',
          'spco2':'\pCOO{} (\\textmu atm)','apco2':'Air \pCOO{} (\\textmu atm)','o2os':'DO (\\textmu mol kg$^{-1}$)',
          'AOUos':'$\Delta$O$_2$ (\\textmu mol kg$^{-1}$)','chlos':'Chl (\\textmu g l$^{-1}$)'}

In [8]:
dft.loc[(dft['modvar']=='phos')&~pd.isnull(dft['obs_SeasAmp']),['datasetID']].values.flatten()

array(['pmel_co2_moorings_c2e7_ecb9_4565',
       'pmel_co2_moorings_ab27_2faf_3aa1',
       'pmel_co2_moorings_ab03_dbac_935b',
       'pmel_co2_moorings_9675_b3d3_c1c1',
       'pmel_co2_moorings_cba8_5413_09f9',
       'pmel_co2_moorings_ec3a_a5d3_70e4'], dtype=object)

In [9]:
#varlist=['tos','sos','phos','spco2','o2os','l10chlos'] # need to add ,'hplus'
Tvec=[7,31,365]
freq='daily'
dfbxf=pd.read_csv(fb.bxfbase+f"bxf_df.{'_'.join([str(el) for el in Tvec])}.{freq}.csv")

In [10]:
dfbxf.keys()

Index(['Unnamed: 0', 'ivar', 'datasetID', 'var_1_7_obs', 'var_7_31_obs',
       'var_31_365_obs', 'var_g365_obs', 'var_total_obs', 'vards_1_7_obs',
       'vards_7_31_obs', 'vards_31_365_obs', 'vards_g365_obs',
       'vards_total_obs', 'var_seas_obs', 'var_b_seas_obs', 'var_b_1_7_obs',
       'var_b_7_31_obs', 'var_b_31_365_obs', 'var_b_g365_obs',
       'var_b_total_obs', 'vards_b_1_7_obs', 'vards_b_7_31_obs',
       'vards_b_31_365_obs', 'vards_b_g365_obs', 'vards_b_total_obs',
       'var_1_7_mod', 'var_7_31_mod', 'var_31_365_mod', 'var_g365_mod',
       'var_total_mod', 'vards_1_7_mod', 'vards_7_31_mod', 'vards_31_365_mod',
       'vards_g365_mod', 'vards_total_mod', 'var_seas_mod', 'var_1_7_obs_N',
       'var_7_31_obs_N', 'var_31_365_obs_N', 'var_g365_obs_N',
       'var_total_obs_N', 'vards_1_7_obs_N', 'vards_7_31_obs_N',
       'vards_31_365_obs_N', 'vards_g365_obs_N', 'vards_total_obs_N',
       'var_seas_obs_N', 'var_b_seas_obs_N', 'var_b_1_7_obs_N',
       'var_b_7_31_obs_N

In [11]:
def formatR(r,p):
    if p<0.05:
        return r"\textbf{"+f"{r:4.2f}"+r"}"
    else:
        return f"{r:13.2f}"

### stats based on seasonal cycle variance/std

In [12]:
# print stats for seas cycle
label_seasAll='Seasonal cycle, all available sites'
rows_seasAll=[]
print('Seasonal Cycle STD, all data')
print(f"{'var':16} {'r':10} {'bias':10} {'RMSE':10} {'nRMSE':10} {'Rsq':10} {'WSS':10} {'N':10} {'p_r':10}")
for ind,mvar in enumerate(varlist):
    iii=(dfbxf['ivar']==mvar)&~pd.isnull(dfbxf['var_seas_obs'])
    x=np.sqrt(dfbxf.loc[iii,['var_seas_obs']].values.flatten())
    y=np.sqrt(dfbxf.loc[iii,['var_seas_mod']].values.flatten())
    r,bias,RMSE,nRMSE,Rsq,WSS,N,rp = getstats(x,y)
    print(f'{mvar:10} {r:10.2f} {bias:10.2f} {RMSE:10.2f} {nRMSE:10.2f} {Rsq:10.2f} {WSS:10.2f} {N:10} {rp:10.2f}')
    rows_seasAll.append(f'{namedict[mvar]:20} & {formatR(r,rp)} & {bias:10.2f} & {bias/np.std(x):10.2f} & {RMSE:10.2f} & {nRMSE:10.2f} & {Rsq:10.2f} & {N:10}')

Seasonal Cycle STD, all data
var              r          bias       RMSE       nRMSE      Rsq        WSS        N          p_r       
tos              0.94       0.35       0.61       0.42       0.89       0.96         30       0.00
sos              0.83      -0.17       0.56       0.75       0.68       0.70         29       0.00
phos             0.86       0.00       0.01       0.53       0.74       0.92          6       0.03
phosC            0.51      -0.01       0.03       0.89       0.26       0.64         19       0.02
spco2            0.55      -8.71      20.78       0.92       0.30       0.65         29       0.00
apco2            0.75      -2.24       2.79       1.23       0.56       0.62         29       0.00
o2os             0.87      -1.67       5.73       0.56       0.76       0.88          7       0.01
AOUos            0.78      -8.39      12.63       1.07       0.61       0.55          7       0.04
chlos            0.59      -0.09       0.16       0.98       0.34       0.

In [13]:
# print stats for seas cycle
label_seaspH='Seasonal cycle, sites with \\sensorpH'
rows_seaspH=[]
print('Seasonal Cycle, sites with ph sensors and ph estimates')
print(f"{'var':16} {'r':10} {'bias':10} {'nbias':10} {'RMSE':10} {'nRMSE':10} {'Rsq':10} {'WSS':10} {'N':10} {'p_r':10}")
dsidsph=list(dfbxf.loc[(dfbxf['ivar']=='phos')&~pd.isnull(dfbxf['var_seas_obs']),['datasetID']].values.flatten())
dsidsphC=list(dfbxf.loc[(dfbxf['ivar']=='phosC')&~pd.isnull(dfbxf['var_seas_obs']),['datasetID']].values.flatten())
locs=set(dsidsph).intersection(set(dsidsphC))
for ind,mvar in enumerate(varlist):
    iii=(dfbxf['ivar']==mvar)&~pd.isnull(dfbxf['var_seas_obs'])&(dfbxf['datasetID'].isin(locs))
    x=np.sqrt(dfbxf.loc[iii,['var_seas_obs']].values.flatten())
    y=np.sqrt(dfbxf.loc[iii,['var_seas_mod']].values.flatten())
    r,bias,RMSE,nRMSE,Rsq,WSS,N,rp = getstats(x,y)
    print(f'{mvar:10} {r:10.2f} {bias:10.2f} {bias/np.std(x):10.2f} {RMSE:10.2f} {nRMSE:10.2f} {Rsq:10.2f} {WSS:10.2f} {N:10} {rp:10.2f}')
    if mvar=='phosC':
        rows_seaspH.append(f'{namedict[mvar]:20} & {formatR(r,rp)} & {bias:10.2f} & {bias/np.std(x):10.2f} & {RMSE:10.2f} & {nRMSE:10.2f} & {Rsq:10.2f} & {N:10}')

Seasonal Cycle, sites with ph sensors and ph estimates
var              r          bias       nbias      RMSE       nRMSE      Rsq        WSS        N          p_r       
tos              0.97       0.25       0.16       0.59       0.38       0.94       0.95          5       0.01
sos              0.87      -0.16      -0.55       0.27       0.93       0.76       0.63          5       0.05
phos             0.85       0.00       0.15       0.01       0.54       0.73       0.92          5       0.07
phosC            0.89       0.01       0.58       0.01       0.81       0.79       0.87          5       0.04
spco2            0.86       4.96       0.32       9.60       0.61       0.75       0.91          5       0.06
apco2            0.10      -3.32      -2.82       3.66       3.11       0.01       0.35          5       0.87
o2os             0.98      -0.83      -0.10       2.79       0.34       0.96       0.96          3       0.13
AOUos            0.94      -5.90      -1.12       6.34     

In [14]:
# print stats for seas cycle
label_seasZ='Seasonal cycle, bathymetric depth$>$1000 m'
rows_seasZ=[]
print('Seasonal Cycle std, Depth>1000m')
print(f"{'var':16} {'r':10} {'bias':10} {'RMSE':10} {'nRMSE':10} {'Rsq':10} {'WSS':10} {'N':10} {'p_r':10}")
for ind,mvar in enumerate(varlist):
    iii=(dfbxf['ivar']==mvar)&~pd.isnull(dfbxf['var_seas_obs'])&(dfbxf['modBathy']>1000)
    x=np.sqrt(dfbxf.loc[iii,['var_seas_obs']].values.flatten())
    y=np.sqrt(dfbxf.loc[iii,['var_seas_mod']].values.flatten())
    r,bias,RMSE,nRMSE,Rsq,WSS,N,rp = getstats(x,y)
    print(f'{mvar:10} {r:10.2f} {bias:10.2f} {RMSE:10.2f} {nRMSE:10.2f} {Rsq:10.2f} {WSS:10.2f} {N:10} {rp:10.2f}')
    rows_seasZ.append(f'{namedict[mvar]:20} & {formatR(r,rp)} & {bias:10.2f} & {bias/np.std(x):10.2f} & {RMSE:10.2f} & {nRMSE:10.2f} & {Rsq:10.2f} & {N:10}')

Seasonal Cycle std, Depth>1000m
var              r          bias       RMSE       nRMSE      Rsq        WSS        N          p_r       
tos              0.94       0.29       0.46       0.48       0.88       0.95         22       0.00
sos              0.87      -0.01       0.07       0.50       0.76       0.93         21       0.00
phos             0.96       0.00       0.01       0.63       0.93       0.94          3       0.17
phosC            0.60      -0.00       0.01       0.82       0.36       0.69         13       0.03
spco2            0.75      -4.52      10.35       0.74       0.56       0.80         21       0.00
apco2            0.78      -1.78       2.34       1.15       0.61       0.62         21       0.00
o2os             0.94      -1.61       7.91       0.65       0.88       0.78          3       0.23
AOUos            0.98     -11.28      17.73       1.05       0.96       0.58          3       0.12
chlos            0.61      -0.11       0.19       1.00       0.37      

### stats based on resid variance/std

In [15]:
# print stats for seas cycle
label_resAll='Residual variability, all available sites'
rows_resAll=[]
print('resid STD, all data')
print(f"{'var':16} {'r':10} {'bias':10} {'RMSE':10} {'nRMSE':10} {'Rsq':10} {'WSS':10} {'N':10} {'p_r':10}")
for ind,mvar in enumerate(varlist):
    iii=(dfbxf['ivar']==mvar)&~pd.isnull(dfbxf['vards_total_obs'])
    x=np.sqrt(dfbxf.loc[iii,['vards_total_obs']].values.flatten())
    y=np.sqrt(dfbxf.loc[iii,['vards_total_mod']].values.flatten())
    r,bias,RMSE,nRMSE,Rsq,WSS,N,rp = getstats(x,y)
    print(f'{mvar:10} {r:10.2f} {bias:10.2f} {RMSE:10.2f} {nRMSE:10.2f} {Rsq:10.2f} {WSS:10.2f} {N:10} {rp:10.2f}')
    rows_resAll.append(f'{namedict[mvar]:20} & {formatR(r,rp)} & {bias:10.2f} & {bias/np.std(x):10.2f} & {RMSE:10.2f} & {nRMSE:10.2f} & {Rsq:10.2f} & {N:10}')

resid STD, all data
var              r          bias       RMSE       nRMSE      Rsq        WSS        N          p_r       
tos              0.73       0.08       0.28       0.87       0.53       0.83         30       0.00
sos              0.91      -0.17       0.31       0.68       0.83       0.81         29       0.00
phos             0.75      -0.02       0.02       1.36       0.56       0.56          6       0.09
phosC            0.87      -0.01       0.02       1.08       0.76       0.56         19       0.00
spco2            0.65     -13.59      18.09       1.26       0.42       0.55         29       0.00
apco2            0.86      -0.64       1.01       0.82       0.74       0.86         29       0.00
o2os             0.60     -11.72      14.03       1.62       0.36       0.46          7       0.15
AOUos            0.80     -13.18      15.21       1.73       0.64       0.46          7       0.03
chlos            0.58      -0.49       0.71       1.25       0.34       0.51       

In [16]:
label_respH='Residual variability, sites with \\sensorpH'
rows_respH=[]
print('resid STD, only sites with ph sensors')
print(f"{'var':16} {'r':10} {'bias':10} {'RMSE':10} {'nRMSE':10} {'Rsq':10} {'WSS':10} {'N':10} {'p_r':10}")
dsidsph=list(dfbxf.loc[(dfbxf['ivar']=='phos')&~pd.isnull(dfbxf['var_seas_obs']),['datasetID']].values.flatten())
for ind,mvar in enumerate(varlist):
    iii=(dfbxf['ivar']==mvar)&~pd.isnull(dfbxf['vards_total_obs'])&(dfbxf['datasetID'].isin(dsidsph))
    x=np.sqrt(dfbxf.loc[iii,['vards_total_obs']].values.flatten())
    y=np.sqrt(dfbxf.loc[iii,['vards_total_mod']].values.flatten())
    r,bias,RMSE,nRMSE,Rsq,WSS,N,rp = getstats(x,y)
    print(f'{mvar:10} {r:10.2f} {bias:10.2f} {RMSE:10.2f} {nRMSE:10.2f} {Rsq:10.2f} {WSS:10.2f} {N:10} {rp:10.2f}')
    if mvar=='phosC':
        rows_respH.append(f'{namedict[mvar]:20} & {formatR(r,rp)} & {bias:10.2f} & {bias/np.std(x):10.2f} & {RMSE:10.2f} & {nRMSE:10.2f} & {Rsq:10.2f} & {N:10}')

resid STD, only sites with ph sensors
var              r          bias       RMSE       nRMSE      Rsq        WSS        N          p_r       
tos              0.98      -0.06       0.08       0.28       0.97       0.98          6       0.00
sos              0.99      -0.10       0.14       0.54       0.99       0.90          6       0.00
phos             0.75      -0.02       0.02       1.36       0.56       0.56          6       0.09
phosC            0.73      -0.01       0.02       1.31       0.53       0.58          5       0.16
spco2            0.80     -14.48      18.41       1.29       0.65       0.57          6       0.05
apco2            0.76      -0.80       1.22       0.98       0.57       0.78          6       0.08
o2os            -0.91     -11.02      11.67       4.11       0.82       0.28          3       0.28
AOUos            0.81     -12.63      12.93       3.62       0.66       0.35          3       0.39
chlos            0.42      -0.52       0.64       1.60       0.17

In [17]:
label_resZ='Residual variability, bathymetric depth$>$1000 m'
rows_resZ=[]
print('resid STD, Depth>1000m')
print(f"{'var':16} {'r':10} {'bias':10} {'RMSE':10} {'nRMSE':10} {'Rsq':10} {'WSS':10} {'N':10} {'p_r':10}")
for ind,mvar in enumerate(varlist):
    iii=(dfbxf['ivar']==mvar)&~pd.isnull(dfbxf['vards_total_obs'])&(dfbxf['modBathy']>1000)
    x=np.sqrt(dfbxf.loc[iii,['vards_total_obs']].values.flatten())
    y=np.sqrt(dfbxf.loc[iii,['vards_total_mod']].values.flatten())
    r,bias,RMSE,nRMSE,Rsq,WSS,N,rp = getstats(x,y)
    print(f'{mvar:10} {r:10.2f} {bias:10.2f} {RMSE:10.2f} {nRMSE:10.2f} {Rsq:10.2f} {WSS:10.2f} {N:10} {rp:10.2f}')
    rows_resZ.append(f'{namedict[mvar]:20} & {formatR(r,rp)} & {bias:10.2f} & {bias/np.std(x):10.2f} & {RMSE:10.2f} & {nRMSE:10.2f} & {Rsq:10.2f} & {N:10}')

resid STD, Depth>1000m
var              r          bias       RMSE       nRMSE      Rsq        WSS        N          p_r       
tos              0.85       0.13       0.26       0.86       0.72       0.86         22       0.00
sos              0.87      -0.06       0.09       0.67       0.76       0.88         21       0.00
phos            -0.97      -0.01       0.01       6.00       0.95       0.11          3       0.14
phosC            0.76      -0.01       0.02       1.00       0.58       0.42         13       0.00
spco2            0.58      -9.85      14.34       1.19       0.33       0.49         21       0.01
apco2            0.78      -0.73       0.99       1.15       0.62       0.77         21       0.00
o2os             0.94     -12.32      16.58       1.29       0.88       0.54          3       0.22
AOUos            0.95     -14.08      17.98       1.41       0.90       0.53          3       0.20
chlos            0.61      -0.47       0.72       1.17       0.37       0.52    

### total variability


In [18]:
# full STD including seasonal cycle
label_totAll='Total variability, all available sites'
rows_totAll=[]
print('Total STD, all data')
print(f"{'var':16} {'r':10} {'bias':10} {'RMSE':10} {'nRMSE':10} {'Rsq':10} {'WSS':10} {'N':10} {'p_r':10}")
for ind,mvar in enumerate(varlist):
    iii=(dfbxf['ivar']==mvar)&~pd.isnull(dfbxf['var_total_obs'])
    x=np.sqrt(dfbxf.loc[iii,['var_total_obs']].values.flatten())
    y=np.sqrt(dfbxf.loc[iii,['var_total_mod']].values.flatten())
    r,bias,RMSE,nRMSE,Rsq,WSS,N,rp = getstats(x,y)
    print(f'{mvar:10} {r:10.2f} {bias:10.2f} {RMSE:10.2f} {nRMSE:10.2f} {Rsq:10.2f} {WSS:10.2f} {N:10} {rp:10.2f}')
    rows_totAll.append(f'{namedict[mvar]:20} & {formatR(r,rp)} & {bias:10.2f} & {bias/np.std(x):10.2f} & {RMSE:10.2f} & {nRMSE:10.2f} & {Rsq:10.2f} & {N:10}')

Total STD, all data
var              r          bias       RMSE       nRMSE      Rsq        WSS        N          p_r       
tos              0.93       0.40       0.66       0.47       0.87       0.95         30       0.00
sos              0.87      -0.25       0.63       0.73       0.76       0.74         29       0.00
phos             0.86      -0.01       0.01       0.64       0.74       0.89          6       0.03
phosC            0.58      -0.02       0.03       0.93       0.34       0.63         19       0.01
spco2            0.60     -16.19      25.37       1.05       0.36       0.63         29       0.00
apco2            0.87      -2.13       2.43       1.05       0.76       0.74         29       0.00
o2os             0.70      -8.81      12.46       1.02       0.49       0.67          7       0.08
AOUos            0.75     -15.56      19.02       1.42       0.56       0.50          7       0.05
chlos            0.55      -0.47       0.70       1.19       0.30       0.53       

In [19]:
label_totpH='Total variability, sites with \\sensorpH'
rows_totpH=[]
print('total STD, only sites with ph sensors')
print(f"{'var':16} {'r':10} {'bias':10} {'RMSE':10} {'nRMSE':10} {'Rsq':10} {'WSS':10} {'N':10} {'p_r':10}")
dsidsph=list(dfbxf.loc[(dfbxf['ivar']=='phos')&~pd.isnull(dfbxf['var_seas_obs']),['datasetID']].values.flatten())
for ind,mvar in enumerate(varlist):
    iii=(dfbxf['ivar']==mvar)&~pd.isnull(dfbxf['var_total_obs'])&(dfbxf['datasetID'].isin(dsidsph))
    x=np.sqrt(dfbxf.loc[iii,['var_total_obs']].values.flatten())
    y=np.sqrt(dfbxf.loc[iii,['var_total_mod']].values.flatten())
    r,bias,RMSE,nRMSE,Rsq,WSS,N,rp = getstats(x,y)
    print(f'{mvar:10} {r:10.2f} {bias:10.2f} {RMSE:10.2f} {nRMSE:10.2f} {Rsq:10.2f} {WSS:10.2f} {N:10} {rp:10.2f}')
    if mvar=='phosC':
        rows_totpH.append(f'{namedict[mvar]:20} & {formatR(r,rp)} & {bias:10.2f} & {bias/np.std(x):10.2f} & {RMSE:10.2f} & {nRMSE:10.2f} & {Rsq:10.2f} & {N:10}')

total STD, only sites with ph sensors
var              r          bias       RMSE       nRMSE      Rsq        WSS        N          p_r       
tos              0.97       0.21       0.50       0.30       0.94       0.97          6       0.00
sos              0.96      -0.20       0.29       0.76       0.93       0.79          6       0.00
phos             0.86      -0.01       0.01       0.64       0.74       0.89          6       0.03
phosC            0.88      -0.00       0.01       0.52       0.78       0.94          5       0.05
spco2            0.89      -6.14      10.27       0.57       0.79       0.91          6       0.02
apco2            0.88      -2.80       2.92       1.67       0.77       0.60          6       0.02
o2os             0.99      -7.42       7.88       2.22       0.98       0.62          3       0.10
AOUos            0.67     -13.74      14.08       3.45       0.44       0.40          3       0.54
chlos            0.28      -0.52       0.64       1.62       0.08

In [20]:
label_totZ='Total variability, bathymetric depth$>$1000 m'
rows_totZ=[]
print('total STD, Depth>1000m')
print(f"{'var':16} {'r':10} {'bias':10} {'RMSE':10} {'nRMSE':10} {'Rsq':10} {'WSS':10} {'N':10} {'p_r':10}")
for ind,mvar in enumerate(varlist):
    iii=(dfbxf['ivar']==mvar)&~pd.isnull(dfbxf['var_total_obs'])&(dfbxf['modBathy']>1000)
    x=np.sqrt(dfbxf.loc[iii,['var_total_obs']].values.flatten())
    y=np.sqrt(dfbxf.loc[iii,['var_total_mod']].values.flatten())
    r,bias,RMSE,nRMSE,Rsq,WSS,N,rp = getstats(x,y)
    print(f'{mvar:10} {r:10.2f} {bias:10.2f} {RMSE:10.2f} {nRMSE:10.2f} {Rsq:10.2f} {WSS:10.2f} {N:10} {rp:10.2f}')
    rows_totZ.append(f'{namedict[mvar]:20} & {formatR(r,rp)} & {bias:10.2f} & {bias/np.std(x):10.2f} & {RMSE:10.2f} & {nRMSE:10.2f} & {Rsq:10.2f} & {N:10}')

total STD, Depth>1000m
var              r          bias       RMSE       nRMSE      Rsq        WSS        N          p_r       
tos              0.92       0.37       0.54       0.60       0.84       0.92         22       0.00
sos              0.86      -0.06       0.11       0.61       0.75       0.90         21       0.00
phos             0.97      -0.00       0.01       1.52       0.94       0.80          3       0.15
phosC            0.50      -0.01       0.02       0.94       0.25       0.50         13       0.09
spco2            0.56     -10.46      17.16       1.05       0.31       0.60         21       0.01
apco2            0.76      -1.90       2.25       1.23       0.58       0.66         21       0.00
o2os             0.89      -9.51      16.16       0.92       0.80       0.62          3       0.30
AOUos            0.98     -17.76      24.02       1.21       0.95       0.57          3       0.14
chlos            0.60      -0.46       0.72       1.12       0.36       0.54    

### chl

In [21]:
#l10chl full std:
print(f"{'var':16} {'r':10} {'bias':10} {'RMSE':10} {'nRMSE':10} {'Rsq':10} {'WSS':10} {'N':10} {'p_r':10}")
ivar='l10chlos'
iii=(dfbxf['ivar']=='l10chlos')&~pd.isnull(dfbxf['var_total_obs'])
x=np.sqrt(dfbxf.loc[iii,['var_total_obs']].values.flatten())
y=np.sqrt(dfbxf.loc[iii,['var_total_mod']].values.flatten())
r,bias,RMSE,nRMSE,Rsq,WSS,N,rp = getstats(x,y)
print(f'{mvar:10} {r:10.2f} {bias:10.2f} {RMSE:10.2f} {nRMSE:10.2f} {Rsq:10.2f} {WSS:10.2f} {N:10} {rp:10.2f}')
#chl full std:
ivar='chlos'
iii=(dfbxf['ivar']=='chlos')&~pd.isnull(dfbxf['var_total_obs'])
x=np.sqrt(dfbxf.loc[iii,['var_total_obs']].values.flatten())
y=np.sqrt(dfbxf.loc[iii,['var_total_mod']].values.flatten())
r,bias,RMSE,nRMSE,Rsq,WSS,N,rp = getstats(x,y)
print(f'{mvar:10} {r:10.2f} {bias:10.2f} {RMSE:10.2f} {nRMSE:10.2f} {Rsq:10.2f} {WSS:10.2f} {N:10} {rp:10.2f}')

var              r          bias       RMSE       nRMSE      Rsq        WSS        N          p_r       
chlos           -0.07      -0.12       0.16       2.37       0.01       0.34         11       0.84
chlos            0.55      -0.47       0.70       1.19       0.30       0.53         11       0.08


In [22]:
#l10chl 31-365:
print(f"{'var':16} {'r':10} {'bias':10} {'RMSE':10} {'nRMSE':10} {'Rsq':10} {'WSS':10} {'N':10} {'p_r':10}")
ivar='l10chlos'
iii=(dfbxf['ivar']=='l10chlos')&~pd.isnull(dfbxf['var_31_365_obs'])
x=np.sqrt(dfbxf.loc[iii,['var_31_365_obs']].values.flatten())
y=np.sqrt(dfbxf.loc[iii,['var_31_365_mod']].values.flatten())
r,bias,RMSE,nRMSE,Rsq,WSS,N,rp = getstats(x,y)
print(f'{mvar:10} {r:10.2f} {bias:10.2f} {RMSE:10.2f} {nRMSE:10.2f} {Rsq:10.2f} {WSS:10.2f} {N:10} {rp:10.2f}')
#chl 31-365:
ivar='chlos'
iii=(dfbxf['ivar']=='chlos')&~pd.isnull(dfbxf['var_31_365_obs'])
x=np.sqrt(dfbxf.loc[iii,['var_31_365_obs']].values.flatten())
y=np.sqrt(dfbxf.loc[iii,['var_31_365_mod']].values.flatten())
r,bias,RMSE,nRMSE,Rsq,WSS,N,rp = getstats(x,y)
print(f'{mvar:10} {r:10.2f} {bias:10.2f} {RMSE:10.2f} {nRMSE:10.2f} {Rsq:10.2f} {WSS:10.2f} {N:10} {rp:10.2f}')

var              r          bias       RMSE       nRMSE      Rsq        WSS        N          p_r       
chlos           -0.08      -0.02       0.10       2.39       0.01       0.31         11       0.81
chlos            0.50      -0.20       0.31       1.14       0.25       0.59         11       0.12


In [23]:
dft['seasdiff']=np.abs(dft.obs_SeasAmp-dft.mod_SeasAmp)

In [24]:
dfbxf['seasdiff']=np.abs(np.sqrt(dfbxf.var_seas_obs)-np.sqrt(dfbxf.var_seas_mod))
dfbxf.loc[(dfbxf.ivar=='phos')|(dfbxf.ivar=='phosC'),['shortTitle','ivar','var_seas_obs','var_seas_mod','seasdiff']].sort_values(by=['seasdiff',],ascending=False)

,shortTitle,ivar,var_seas_obs,var_seas_mod,seasdiff
250,Kodiak,phosC,0.013791,0.000462,0.095930
229,Cha ba,phosC,0.004626,0.000438,0.047093
22,Gulf of Maine,phosC,0.000979,0.002483,0.018537
17,Gulf of Maine,phos,0.001069,0.002483,0.017125
216,Cape Elizabeth,phosC,0.002480,0.001141,0.016020
79,Coastal MS,phosC,0.003297,0.001727,0.015870
174,CCE2,phosC,0.000413,0.001309,0.015851
62,Gray's Reef,phos,0.004681,0.003053,0.013159
302,KEO,phosC,0.000911,0.001601,0.009837
297,KEO,phos,0.000939,0.001601,0.009366


In [25]:
dfbxf['seasdiff']=np.abs(dfbxf.var_seas_obs-dfbxf.var_seas_mod)
dfbxf.loc[(dfbxf.ivar=='spco2'),['shortTitle','ivar','var_seas_obs','var_seas_mod','seasdiff']].sort_values(by=['seasdiff',],ascending=False)

,shortTitle,ivar,var_seas_obs,var_seas_mod,seasdiff
249,Kodiak,spco2,10439.189395,358.250154,10080.939241
244,GAKOA,spco2,3681.380326,384.716374,3296.663952
103,Cheeca Rocks,spco2,3370.113106,669.548674,2700.564432
225,Cha ba,spco2,2652.570783,372.969339,2279.601444
48,Crescent Reef,spco2,2160.678779,999.989511,1160.689269
338,CRIMP2,spco2,1133.428841,147.936929,985.491912
170,CCE2,spco2,505.465987,1464.612440,959.146453
78,Coastal MS,spco2,1480.187394,653.647596,826.539798
18,Gulf of Maine,spco2,1220.057521,2025.187041,805.129520
43,Hog Reef,spco2,1803.627069,999.989511,803.637558


In [26]:
dfbxf['seasdiff']=np.abs(dfbxf.var_seas_obs-dfbxf.var_seas_mod)
dfbxf.loc[(dfbxf.ivar=='sos'),['shortTitle','ivar','var_seas_obs','var_seas_mod','seasdiff']].sort_values(by=['seasdiff',],ascending=False)

,shortTitle,ivar,var_seas_obs,var_seas_mod,seasdiff
76,Coastal MS,sos,14.111176,1.510693,12.600483
243,GAKOA,sos,3.829591,0.374560,3.455031
213,Cape Elizabeth,sos,0.321099,0.999829,0.678729
16,Gulf of Maine,sos,0.574951,0.057276,0.517675
61,Gray's Reef,sos,0.376169,0.087217,0.288952
101,Cheeca Rocks,sos,0.221211,0.017002,0.204209
127,La Parguera,sos,0.284970,0.093983,0.190986
393,Chuuk,sos,0.018123,0.091402,0.073279
47,Crescent Reef,sos,0.028106,0.004159,0.023948
223,Cha ba,sos,0.325153,0.346461,0.021308


### Write table file

In [27]:
lines=[]# list of lines in table
lines.append(r"""
    \begin{longtable}{l c c c c c c c}
    \caption{\tblStatsCaption}
    \label{tbl:Stats} \\
    \hline 
    Variable & R & Bias & nBias & RMSE & nRMSE & R$^2$ & N \\
   \hline
\endfirsthead
\multicolumn{8}{c}%
    {\tablename\ \thetable\ -- \textit{Continued from previous page}} \\
    \hline
    Variable & R & Bias & nBias & RMSE & nRMSE & R$^2$ & N \\
   \hline
\endhead
    \hline \multicolumn{8}{r}{\textit{Continued on next page}} \\
    \endfoot
    \hline
    \endlastfoot
    """)

In [28]:
def formatDescript(fff):
    #return r"\multicolumn{8}{l}{"+f"{fff}"+r"} \\ "+"\n"
    return r"\multicolumn{8}{l}{"+r"\textsc{"+f"{fff}"+r"}} \\ "+"\n"
def formatLines(fff,lines):
    for el in fff:
        lines.append(el+r" \\"+"\n")

In [29]:
lines.append(formatDescript(label_seasAll))
formatLines(rows_seasAll,lines)
lines.append(formatDescript(label_seaspH))
formatLines(rows_seaspH,lines)
lines.append(formatDescript(label_seasZ))
formatLines(rows_seasZ,lines)

In [30]:
lines.append(formatDescript(label_resAll))
formatLines(rows_resAll,lines)
lines.append(formatDescript(label_respH))
formatLines(rows_respH,lines)
lines.append(formatDescript(label_resZ))
formatLines(rows_resZ,lines)

In [31]:
lines.append(formatDescript(label_totAll))
formatLines(rows_totAll,lines)
lines.append(formatDescript(label_totpH))
formatLines(rows_totpH,lines)
lines.append(formatDescript(label_totZ))
formatLines(rows_totZ,lines)

In [32]:
lines.append(r"""
    \end{longtable}
    """)

In [33]:
f = open('TableStats.tex', 'w')
f.writelines(lines)
f.close()